In [10]:
from birdcall.data import *

import pandas as pd
import librosa
import matplotlib.pyplot as plt
import soundfile as sf
from torch.utils.data import Dataset
import numpy as np
from pathlib import Path

In [2]:
recs = pd.read_pickle('data/recs.pkl')

In [2]:
from pyfftw.builders import rfft as rfft_builder
from pyfftw import empty_aligned

mel_bands=80
mel_min=27.5
mel_max=10000

In [3]:
def spectrogram(samples, sample_rate, frame_len, fps, batch=48, dtype=None,
                bins=None, plans=None):
    """
    Computes a magnitude spectrogram for a given vector of samples at a given
    sample rate (in Hz), frame length (in samples) and frame rate (in Hz).
    Allows to transform multiple frames at once for improved performance (with
    a default value of 48, more is not always better). Returns a numpy array.
    Allows to return a limited number of bins only, with improved performance
    over discarding them afterwards. Optionally accepts a set of precomputed
    plans created with spectrogram_plans(), required when multi-threading.
    """
    if dtype is None:
        dtype = samples.dtype
    if bins is None:
        bins = frame_len // 2 + 1
    if len(samples) < frame_len:
        return np.empty((0, bins), dtype=dtype)
    if plans is None:
        plans = spectrogram_plans(frame_len, batch, dtype)
    rfft1, rfft, win = plans
    hopsize = int(sample_rate // fps)
    num_frames = (len(samples) - frame_len) // hopsize + 1
    nabs = np.abs
    naa = np.asanyarray
    if batch > 1 and num_frames >= batch and samples.flags.c_contiguous:
        frames = np.lib.stride_tricks.as_strided(
                samples, shape=(num_frames, frame_len),
                strides=(samples.strides[0] * hopsize, samples.strides[0]))
        spect = [nabs(rfft(naa(frames[pos:pos + batch:], dtype) * win)[:, :bins])
                 for pos in range(0, num_frames - batch + 1, batch)]
        samples = samples[(num_frames // batch * batch) * hopsize::]
        num_frames = num_frames % batch
    else:
        spect = []
    if num_frames:
        spect.append(np.vstack(
                [nabs(rfft1(naa(samples[pos:pos + frame_len:],
                                dtype) * win)[:bins:])
                 for pos in range(0, len(samples) - frame_len + 1, hopsize)]))
    return np.vstack(spect) if len(spect) > 1 else spect[0]


def create_mel_filterbank(sample_rate, frame_len, num_bands, min_freq,
                          max_freq):
    """
    Creates a mel filterbank of `num_bands` triangular filters, with the first
    filter starting at `min_freq` and the last one stopping at `max_freq`.
    Returns the filterbank as a matrix suitable for a dot product against
    magnitude spectra created from samples at a sample rate of `sample_rate`
    with a window length of `frame_len` samples.
    """
    # prepare output matrix
    input_bins = (frame_len // 2) + 1
    filterbank = np.zeros((input_bins, num_bands))

    # mel-spaced peak frequencies
    min_mel = 1127 * np.log1p(min_freq / 700.0)
    max_mel = 1127 * np.log1p(max_freq / 700.0)
    spacing = (max_mel - min_mel) / (num_bands + 1)
    peaks_mel = min_mel + np.arange(num_bands + 2) * spacing
    peaks_hz = 700 * (np.exp(peaks_mel / 1127) - 1)
    fft_freqs = np.linspace(0, sample_rate / 2., input_bins)
    peaks_bin = np.searchsorted(fft_freqs, peaks_hz)

    # fill output matrix with triangular filters
    for b, filt in enumerate(filterbank.T):
        # The triangle starts at the previous filter's peak (peaks_freq[b]),
        # has its maximum at peaks_freq[b+1] and ends at peaks_freq[b+2].
        left_hz, top_hz, right_hz = peaks_hz[b:b + 3]  # b, b+1, b+2
        left_bin, top_bin, right_bin = peaks_bin[b:b + 3]
        # Create triangular filter compatible to yaafe
        filt[left_bin:top_bin] = ((fft_freqs[left_bin:top_bin] - left_hz) /
                                  (top_bin - left_bin))
        filt[top_bin:right_bin] = ((right_hz - fft_freqs[top_bin:right_bin]) /
                                   (right_bin - top_bin))
        filt[left_bin:right_bin] /= filt[left_bin:right_bin].sum()

    return filterbank

def spectrogram_plans(frame_len, batch=48, dtype=np.float32):
    """
    Precompute plans for spectrogram(), for a given frame length, batch size
    and dtype. Returns two plans (single spectrum and batch), and a window.
    """
    input_array = empty_aligned((batch, frame_len), dtype=dtype)
    win = np.hanning(frame_len).astype(dtype)
    return (rfft_builder(input_array[0]), rfft_builder(input_array), win)

Pulling it all together into a datset

In [4]:
#export data

filterbank = create_mel_filterbank(SAMPLE_RATE, 256, mel_bands, mel_min, mel_max)
def audio_to_melspec(audio):
    spec = spectrogram(audio, SAMPLE_RATE, 256, 128)
    return (spec @ filterbank).T

In [6]:
#export data

from collections import defaultdict
from multiprocessing import Pool
import torch

def create_example(item):
    cls_idx, path, num_specs = item
    x, _ = sf.read(path)

    example_duration = num_specs * 5 * SAMPLE_RATE
    if x.shape[0] < example_duration:
        x = np.tile(x, example_duration // x.shape[0] + 1)
    
    start_frame = np.random.randint(0, x.shape[0] - example_duration+1)
    x = x[start_frame:start_frame+example_duration]

    xs = []
    for i in range(num_specs):
        for j in range(3):
            start_frame = int((i * 3 + j) * 1.66 * SAMPLE_RATE)
            xs.append(x[start_frame:start_frame+int(1.66*SAMPLE_RATE)])

    specs = []
    for x in xs:
        specs.append(audio_to_melspec(x))
    specs = np.stack(specs)
    imgs = specs.reshape(-1, 3, 80, 212)

    one_hot = np.zeros((264))
    one_hot[cls_idx] = 1

    return imgs.astype(np.float32), one_hot

def bin_items(recs, classes):
    binned_items = defaultdict(list)
    for key in recs.keys():
        for path, duration in recs[key]:
            if duration < 7.5: binned_items[1].append((classes.index(key), path, 1))
            elif duration < 12.5: binned_items[2].append((classes.index(key), path, 2))
            elif duration < 25: binned_items[4].append((classes.index(key), path, 4))
            elif duration < 45: binned_items[6].append((classes.index(key), path, 6))
            else: binned_items[10].append((classes.index(key), path, 10))
    return binned_items

class MelspecShortishDataset(torch.utils.data.Dataset):
    def __init__(self, recs, classes):
        self.recs = recs
        self.classes = classes
        self.binned_items = bin_items(recs, classes)
        
    def __len__(self): raise(NotImplementedError)       

    def __getitem__(self, bin_num):
        item_idx = np.random.randint(0, len(self.binned_items[bin_num]))
        item = self.binned_items[bin_num][item_idx]
        return create_example(item)
    
def batch_sampler(batch_size=16, len_mult=1):
    for i in range(len_mult * 264 // batch_size):
        chosen_bin = np.random.choice([1,2,4,6,10], p=[0.1, 0.225, 0.225, 0.225, 0.225])
        yield [chosen_bin] * batch_size

class BatchSampler():
    def __init__(self, batch_size=16, len_mult=1):
        self.batch_size = batch_size
        self.len_mult = len_mult
    
    def __iter__(self):
        return batch_sampler(self.batch_size, self.len_mult)

    def __len__(self):
        return self.len_mult * 264 // self.batch_size

In [16]:
train_ds = MelspecShortishDataset(pd.read_pickle('data/train_set.pkl'), pd.read_pickle('data/classes.pkl'))
train_dl = torch.utils.data.DataLoader(train_ds, batch_sampler=BatchSampler(), num_workers=NUM_WORKERS)

In [35]:
%%time
for batch in train_dl:
    pass

CPU times: user 24 ms, sys: 224 ms, total: 248 ms
Wall time: 1.63 s


In [64]:
#export data

class MelspecShortishValidatioDataset(torch.utils.data.Dataset):
    def __init__(self, items, classes):
        self.classes = classes
        self.items = items
                
    def __len__(self): return len(self.items)
    
    def __getitem__(self, idx):
        return self.create_example(self.items[idx])
    
    def create_example(self, item):
        cls_idx, path, num_specs = item
        if path.name.split('.')[1] == 'wav':
            x, _ = sf.read(path)

            example_duration = num_specs * 5 * SAMPLE_RATE
            if x.shape[0] < example_duration:
                x = np.tile(x, example_duration // x.shape[0] + 1)

            start_frame = 0
            x = x[start_frame:example_duration]

            xs = []
            for i in range(num_specs):
                for j in range(3):
                    start_frame = int((i * 3 + j) * 1.66 * SAMPLE_RATE)
                    xs.append(x[start_frame:start_frame+int(1.66*SAMPLE_RATE)])

            specs = []
            for x in xs:
                specs.append(audio_to_melspec(x))
        else: # .npy
            frames_per_spec = 212
            x = np.load(path)
            example_duration = num_specs * 3 * frames_per_spec
            
            if x.shape[1] < example_duration:
                x = np.tile(x, (example_duration // x.shape[1] + 1))
                
            specs = []
            for i in range(num_specs):
                for j in range(3):
                    start_frame = int((i * 3 + j) * frames_per_spec)
                    specs.append(x[:, start_frame:start_frame+frames_per_spec])
            
        specs = np.stack(specs)
        imgs = specs.reshape(-1, 3, 80, 212)

        one_hot = np.zeros((264))
        one_hot[cls_idx] = 1

        return imgs.astype(np.float32), one_hot

In [65]:
val_items = bin_items(recs, pd.read_pickle('data/classes.pkl'))

In [66]:
for num_specs in val_items.keys():
    valid_ds = MelspecShortishValidatioDataset(val_items[num_specs], pd.read_pickle('data/classes.pkl'))
    valid_dl = torch.utils.data.DataLoader(valid_ds, num_workers=NUM_WORKERS, pin_memory=True)

###  Saving spectrograms to disk to speed up training

In [ ]:
%%time

for directory in Path('data/train_resampled/').iterdir():
    ebird_code = directory.name
    for recording in directory.iterdir():
        if recording.stem in ['XC487556', 'XC487557', 'XC246425']: continue

        !mkdir -p data/npy/train_resampled/{ebird_code}
        x = sf.read(recording)[0]
        x = audio_to_melspec(x).astype(np.float32)
        np.save(f'data/npy/train_resampled/{ebird_code}/{recording.stem}.npy', x)

In [8]:
mkdir -p data/npy/shifted

In [13]:
%%time

for recording in Path('data/shifted/').iterdir():
    x = sf.read(recording)[0]
    x = audio_to_melspec(x).astype(np.float32)
    np.save(f'data/npy/shifted/{recording.stem}.npy', x)

CPU times: user 22.4 s, sys: 2.86 s, total: 25.3 s
Wall time: 15.1 s
